In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import gymnasium as gym
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
import gymnasium as gym
import numpy as np
from sapien.core import Pose
import matplotlib.pyplot as plt
import os
import warnings
import shutil
import random
from plot_utils import *

from utils import *
from agent_env_utils import *
from config import color_maps, background_quadrants


In [18]:
def create_directories(data_dir, empty=False):
    final_dir = f"{data_dir}/final"
    os.makedirs(final_dir, exist_ok=True)
    if empty:
        shutil.rmtree(final_dir)
        os.makedirs(final_dir, exist_ok=True)
    return final_dir

def create_config(obj_type, color, size_range, position_range, static=False):
    return {    
        'size': np.random.uniform(*size_range),
        'color': color,
        'obj_type': obj_type,
        'ranges': position_range,
        'static': static
    }
    
def finalize_quadrant(quadrant, scale=1):
    lim_x = quadrant[0]
    lim_y = quadrant[1]
    interval_x = [0, lim_x]
    interval_y = [0, lim_y]
    interval_x.sort()
    interval_y.sort()
    interval_x = np.array(interval_x)*scale
    interval_y = np.array(interval_y)*scale
    return interval_x, interval_y

# Define directories
data_dir = "../benchmark/spatial/try4"
final_dir = create_directories(data_dir, empty=True)

# Initialize pair of colors
color_keys = color_maps.keys()
color_pairs = list(itertools.permutations(color_keys, 2))
random.seed(42)
random.shuffle(color_pairs)

# Define ranges and object parameters
object_shapes = ["sphere", "cube"]
object_size_ranges = {
    "sphere": (0.05, 0.1),
    "cube": (0.05, 0.08),
    "random_sphere": (0.01, 0.05)
}
object_shapes = ["sphere", "cube"]
object_position_range = [(-0.3, 0.3), (-0.3, 0.25)]
num_random_balls = 5
background = 0

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")
    directions = list(env.direction_placement_map.keys())
    

    # Print available directions
    print(f"Available directions: {env.direction_placement_map.keys()}")
    outputs = []
    for background in trange(4):
        for quadrant in background_quadrants[background]:
            #decide the boundary of each background
            interval_x, interval_y = finalize_quadrant(quadrant, scale=0.6)
            object_position_range = [interval_x, interval_y]
            
            for color_idx in range(10): #range(len(color_pairs)):
                color_1, color_2 = color_pairs[color_idx]
                for r in range(1):
                    
                    for obj1, obj2 in list(itertools.permutations(object_shapes, 2)):
                        
                        obj1_config = create_config(obj1, color_maps[color_1], object_size_ranges[obj1], object_position_range, static=True)
                        obj2_config = create_config(obj2, color_maps[color_2], object_size_ranges[obj2], object_position_range, static=True)
                        # sphere1_config = create_config("sphere", color_maps[color_2], object_size_ranges["sphere"], object_position_range, static=True)
                        # random_ball_configs = [ create_config ("sphere", color_maps["purple"], object_size_ranges["random_sphere"], [(-0.5, 0.3), (-0.5, 0.4)])
                        #                     for _ in range(num_random_balls)]

                        #configs = [sphere1_config, cube2_config] + random_ball_configs
                        configs = [obj1_config, obj2_config]
                        configs_collate = collate_infos(configs)

                        env.register_configures(configs_collate)
                        
                        _ = env.reset(options={"reconfigure": True})

                        env.initialize_objects(background=background)
                        obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))

                        initial_poses = env.get_important_obj_poses(mode="all")

                        for i, direction in enumerate(directions):
                            if direction not in env.direction_placement_map:
                                print(f"Skipping invalid direction: {direction}")
                                continue

                            env.set_poses(initial_poses)
                            
                            env.place_cubes_in_direction([(0, 1, direction)])

                            obs, _, _, _, _ = env.step(np.zeros(len(env.action_space.sample())))
                            poses_initial = env.get_important_obj_poses()

                            save_dir_final = f"{final_dir}/{r}_{color_1}_{direction}_{color_2}_bg{background}.png"
                            collect_and_save(env, save_dir_final, steps=1, mode=[f"front{background}", f"side{background}", f"top{background}"])
                            poses_final = env.get_important_obj_poses()

                            bad_case = any(map(lambda x: check_move(x[0], x[1]), zip(poses_initial, poses_final)))
                            if bad_case:
                                print(f"Bad initialization on the {direction} direction, in {color_idx} iter")
                                print(f"Obj1 pose initial {poses_initial[0]}")
                                print(f"Obj2 pose initial {poses_initial[1]}")

                            outputs.append({
                                "source": save_dir_final,
                                "moving_obj": color_1,
                                "direction": direction,
                                "ref_obj": color_2,
                                "bad_case": bad_case,
                                "round": r,
                                "sizes": configs_collate['size'],
                                "poses_final": [np.round(item, 4).tolist() for item in poses_final],
                            })

    write_out(f"{data_dir}/index.jsonl", outputs)

del env

[2024-08-29 15:12:40.120] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.


Available directions: dict_keys(['behind', 'front', 'left', 'right', 'top'])


100%|██████████| 4/4 [22:21<00:00, 335.26s/it]


In [13]:
object_shapes = ["sphere", "cube"]

list(itertools.permutations(object_shapes, 2))

[('sphere', 'cube'), ('cube', 'sphere')]